In [ ]:
#%pip install eloquent-tensorflow embedded_window
#%pip install pandas
#%pip install numpy
#%pip install seaborn
#%pip install -U seaborn-image
#%pip install matplotlib.pyplot
#%pip install scikit-learn

In [ ]:
"""
Get X and y data
(replace with your own)
"""
import numpy as np
import pandas as pd
from embedded_window import Window
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler

window_size = 60
window_shift = 16
window = Window(length=window_size, shift=window_shift)


def read_file(filename, plot: bool = False):
  df = pd.read_csv(filename)
  
  emg = df.iloc[:, 0:64]
    
  if plot:
    emg.plot(title=filename, legend=False)

  # normalize data
  # in this case, accel is in the range -3, 3
  # you have to adapt this line to your own data!
  emg_norm = emg.to_numpy()
  #emg_norm = np.clip(emg.to_numpy(), -10, 10)
  
  y = np.zeros(len(emg_norm))
  
  # apply window
  # returns a (N, window_length, accel.columns) array
  accel_windows, _, _ = window.fit_transform(emg_norm, y)
  return accel_windows


def get_Xy():
  """
  Read files
  One-hot encode y
  """
  x_fist = read_file('./LSTM-Daten/Fist.csv')
  x_thumbs = read_file('./LSTM-Daten/Thumbs.csv')
  x_index = read_file('./LSTM-Daten/Index.csv')
  x_all = read_file('./LSTM-Daten/All.csv')

  y_fist = [0] * len(x_fist)
  y_thumbs = [1] * len(x_thumbs)
  y_index = [2] * len(x_index)
  y_all = [3] * len(x_all)

  X = np.vstack((x_fist, x_thumbs, x_index, x_all))
  y = np.concatenate((y_fist, y_thumbs, y_index, y_all))
  
  num_classes = y.max()
  
  eye = np.eye(num_classes + 1)
  y_hot = np.asarray([eye[yi] for yi in y], dtype=int)

  return X, y_hot


X, y = get_Xy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.3)

num_classes = y.shape[1]

print("X.shape", X.shape)
print("Y.shape", y.shape)

In [ ]:
"""
Print samples of each class
"""
def print_sample(variable_name: str, filename: str):
  sample = read_file(filename, plot=False)[0].flatten()  
  values = ', '.join('%.4f' % x for x in sample)
  print(f'float {variable_name}[{len(sample)}] = {{ {values} }};')

 
print_sample('fist', './LSTM-Daten/Fist.csv')
print_sample('thumbs', './LSTM-Daten/Thumbs.csv')
print_sample('index', './LSTM-Daten/Index.csv')
print_sample('all', './LSTM-Daten/All.csv')

In [ ]:
"""
Instantiate NN for classification
Replace with your own topology
"""
import tensorflow as tf
from tensorflow.keras import layers


def instantiate_lstm_for_classification(input_shape, num_classes):
  model = tf.keras.Sequential()
  model.add(layers.Input(shape=input_shape, batch_size=16))
  model.add(layers.LSTM(12, unroll=False, return_sequences=True))
  model.add(layers.LSTM(12, unroll=False))
  model.add(layers.Dense(64, activation='relu'))
  model.add(layers.Dense(num_classes, activation='softmax'))
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

  return model


model = instantiate_lstm_for_classification(X.shape[1:], num_classes)
print(model.summary())

In [ ]:
"""
Train model
"""
history = model.fit(X_train, y_train, epochs=64, validation_data=(X_val, y_val))

In [ ]:
"""
Evaluate on test data
"""
model.evaluate(X_test, y_test)

In [ ]:
from eloquent_tensorflow import convert_model

#print(convert_model(model))